In [1]:
import numpy as np

data = open('HP1.txt','r', encoding="utf8").read();
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print(data_size,", ",vocab_size)

char_to_ix = {ch:i for i, ch in enumerate(chars)}
ix_to_char = {i:ch for i,ch in enumerate(chars)}
print(char_to_ix)
print(ix_to_char)

def encode(idx,num_entry):
    ret = np.zeros(num_entry)
    ret[idx] = 1
    return ret;

def encode_array(array,num_entry):
    xs = np.zeros((len(array),num_entry))
    for i in range(len(array)):
        xs[i][array[i]] = 1; 
    return xs;

431677 ,  79
{'"': 0, 'V': 1, 'D': 2, 'q': 3, 'c': 4, 'n': 5, 'u': 6, 'F': 7, 'E': 8, 'h': 9, 'v': 10, 'H': 11, ';': 12, '3': 13, ' ': 14, '-': 15, ')': 16, '~': 17, '2': 18, 'M': 19, '\\': 20, 'G': 21, '0': 22, 'g': 23, 'K': 24, 'k': 25, 'd': 26, 'z': 27, 'b': 28, 'e': 29, 'W': 30, 'm': 31, 's': 32, 'B': 33, 'Z': 34, ':': 35, 'r': 36, "'": 37, 'j': 38, '?': 39, 'C': 40, 'J': 41, 'L': 42, 'A': 43, ',': 44, '\t': 45, '9': 46, 'X': 47, 'f': 48, 'U': 49, '.': 50, '\n': 51, 'o': 52, 'N': 53, 'Y': 54, '6': 55, 'y': 56, 'O': 57, 'i': 58, 'p': 59, '1': 60, '5': 61, 'I': 62, 'Q': 63, '4': 64, '*': 65, 't': 66, 'T': 67, '7': 68, '8': 69, 'S': 70, 'P': 71, 'x': 72, 'R': 73, 'a': 74, '(': 75, 'l': 76, 'w': 77, '!': 78}
{0: '"', 1: 'V', 2: 'D', 3: 'q', 4: 'c', 5: 'n', 6: 'u', 7: 'F', 8: 'E', 9: 'h', 10: 'v', 11: 'H', 12: ';', 13: '3', 14: ' ', 15: '-', 16: ')', 17: '~', 18: '2', 19: 'M', 20: '\\', 21: 'G', 22: '0', 23: 'g', 24: 'K', 25: 'k', 26: 'd', 27: 'z', 28: 'b', 29: 'e', 30: 'W', 31: 'm', 32

In [2]:
from RNN import *

seq_length,position = 75,0
inputs = [char_to_ix[ch] for ch in data[position:position+seq_length]]
print(data[position:position+seq_length])
print("inputs",inputs)

targets = [char_to_ix[ch] for ch in data[position+1:position+seq_length+1]] 
print(data[position+1:position+seq_length+1])
print("targets",targets)

n,position = 0,0;
epoch = 20*1000;
lenIn, lenOut, lenRec = vocab_size,vocab_size, seq_length
sizeHidden, numHiddenLayer = 100,1;
learningRate = 0.1;


R = basicRNN(lenIn, lenOut, lenRec, sizeHidden, encode_array(inputs,vocab_size),targets, learningRate)

# training
while n<epoch:
    
    if(position+seq_length+1 >= len(data) or n == 0):
        print("!!!!",len(data))
        position = 0;
        
    inputs  = [char_to_ix[ch] for ch in data[position:position+seq_length]]
    targets = [char_to_ix[ch] for ch in data[position+1:position+seq_length+1]] 

    R.update_inputs_targets(encode_array(inputs,vocab_size),targets)
    R.fwd_pass();
    
    err = R.bwd_pass();
    
    if(n%500 == 0):
        print(n,"err:",err)
        seed = encode(n % vocab_size,vocab_size)
        print(ix_to_char[n % vocab_size])
        result = [];
        R.h  = np.zeros(sizeHidden)
        R.c  = np.zeros(sizeHidden)
        for i in range(40):
            ret = R.inference(seed)
            #print(i,":",ret)
            result.append(ret)
            seed = encode(ret,vocab_size)
        decode = ''.join([ix_to_char[ch] for ch in result] )
        print(decode)

    position += seq_length;
    n += 1;

Harry Potter and the Sorcerer's Stone
CHAPTER ONE
THE BOY WHO LIVED
Mr. and
inputs [11, 74, 36, 36, 56, 14, 71, 52, 66, 66, 29, 36, 14, 74, 5, 26, 14, 66, 9, 29, 14, 70, 52, 36, 4, 29, 36, 29, 36, 37, 32, 14, 70, 66, 52, 5, 29, 51, 40, 11, 43, 71, 67, 8, 73, 14, 57, 53, 8, 51, 67, 11, 8, 14, 33, 57, 54, 14, 30, 11, 57, 14, 42, 62, 1, 8, 2, 51, 19, 36, 50, 14, 74, 5, 26]
arry Potter and the Sorcerer's Stone
CHAPTER ONE
THE BOY WHO LIVED
Mr. and 
targets [74, 36, 36, 56, 14, 71, 52, 66, 66, 29, 36, 14, 74, 5, 26, 14, 66, 9, 29, 14, 70, 52, 36, 4, 29, 36, 29, 36, 37, 32, 14, 70, 66, 52, 5, 29, 51, 40, 11, 43, 71, 67, 8, 73, 14, 57, 53, 8, 51, 67, 11, 8, 14, 33, 57, 54, 14, 30, 11, 57, 14, 42, 62, 1, 8, 2, 51, 19, 36, 50, 14, 74, 5, 26, 14]
!!!! 431677
0 err: -4.369447852467028
"
                                        
500 err: -20.10556071868389
d
ennnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnn
1000 err: -16.647854381587482
o
 wwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwww
1500 err: -26.09930267310883

C:\Users\peter\Anaconda3\study\RNN.py:11: RuntimeWarning: overflow encountered in exp
  return np.exp(array)/ np.sum(np.exp(array)) # return an array
C:\Users\peter\Anaconda3\study\RNN.py:11: RuntimeWarning: invalid value encountered in true_divide
  return np.exp(array)/ np.sum(np.exp(array)) # return an array


ERR! 671.5550351585348   49658.66584218462
ERR! 671.5550351585348   49658.66584218462
ERR! 671.5550351585348   49658.66584218462
ERR! 671.5550351585348   49658.66584218462
ERR! 671.5550351585348   49658.66584218462
ERR! 671.5550351585348   49658.66584218462
ERR! 671.5550351585348   49658.66584218462
ERR! 671.5550351585348   49658.66584218462
ERR! 671.5550351585348   49658.66584218462
ERR! 671.5550351585348   49658.66584218462
ERR! 671.5550351585348   49658.66584218462
ERR! 671.5550351585348   49658.66584218462
ERR! 671.5550351585348   49658.66584218462
ERR! 671.5550351585348   49658.66584218462
ERR! 671.5550351585348   49658.66584218462
ERR! 671.5550351585348   49658.66584218462
ERR! 671.5550351585348   49658.66584218462
ERR! 671.5550351585348   49658.66584218462
ERR! 671.5550351585348   49658.66584218462
ERR! 671.5550351585348   49658.66584218462
ERR! 671.5550351585348   49658.66584218462
ERR! 671.5550351585348   49658.66584218462
ERR! 671.5550351585348   49658.66584218462
ERR! 671.55

ERR! 671.5550351585348   49658.66584218462
ERR! 671.5550351585348   49658.66584218462
ERR! 671.5550351585348   49658.66584218462
ERR! 671.5550351585348   49658.66584218462
ERR! 671.5550351585348   49658.66584218462
ERR! 671.5550351585348   49658.66584218462
ERR! 671.5550351585348   49658.66584218462
ERR! 671.5550351585348   49658.66584218462
ERR! 671.5550351585348   49658.66584218462
ERR! 671.5550351585348   49658.66584218462
ERR! 671.5550351585348   49658.66584218462
ERR! 671.5550351585348   49658.66584218462
ERR! 671.5550351585348   49658.66584218462
ERR! 671.5550351585348   49658.66584218462
ERR! 671.5550351585348   49658.66584218462
ERR! 671.5550351585348   49658.66584218462
ERR! 671.5550351585348   49658.66584218462
ERR! 671.5550351585348   49658.66584218462
ERR! 671.5550351585348   49658.66584218462
ERR! 671.5550351585348   49658.66584218462
ERR! 671.5550351585348   49658.66584218462
ERR! 671.5550351585348   49658.66584218462
ERR! 671.5550351585348   49658.66584218462
ERR! 671.55

ERR! 671.5550351585348   49658.66584218462
ERR! 671.5550351585348   49658.66584218462
ERR! 671.5550351585348   49658.66584218462
ERR! 671.5550351585348   49658.66584218462
ERR! 671.5550351585348   49658.66584218462
ERR! 671.5550351585348   49658.66584218462
ERR! 671.5550351585348   49658.66584218462
ERR! 671.5550351585348   49658.66584218462
ERR! 671.5550351585348   49658.66584218462
ERR! 671.5550351585348   49658.66584218462
ERR! 671.5550351585348   49658.66584218462
ERR! 671.5550351585348   49658.66584218462
ERR! 671.5550351585348   49658.66584218462
ERR! 671.5550351585348   49658.66584218462
ERR! 671.5550351585348   49658.66584218462
ERR! 671.5550351585348   49658.66584218462
ERR! 671.5550351585348   49658.66584218462
ERR! 671.5550351585348   49658.66584218462
ERR! 671.5550351585348   49658.66584218462
ERR! 671.5550351585348   49658.66584218462
ERR! 671.5550351585348   49658.66584218462
ERR! 671.5550351585348   49658.66584218462
ERR! 671.5550351585348   49658.66584218462
ERR! 671.55

...

ERR! 668.2414607646324   49658.666124994445
ERR! 668.2414607646324   49658.666124994445
ERR! 668.2414607646324   49658.666124994445
ERR! 668.2414607646324   49658.666124994445
ERR! 668.2414607646324   49658.666124994445
ERR! 668.2414607646324   49658.666124994445
ERR! 668.2414607646324   49658.666124994445
ERR! 668.2414607646324   49658.666124994445
ERR! 668.2414607646324   49658.666124994445
ERR! 668.2414607646324   49658.666124994445
ERR! 668.2414607646324   49658.666124994445
ERR! 668.2414607646324   49658.666124994445
ERR! 668.2414607646324   49658.666124994445
ERR! 668.2414607646324   49658.666124994445
ERR! 668.2414607646324   49658.666124994445
ERR! 668.2414607646324   49658.666124994445
ERR! 668.2414607646324   49658.666124994445
ERR! 668.2414607646324   49658.666124994445
ERR! 668.2414607646324   49658.666124994445
ERR! 668.2414607646324   49658.666124994445
ERR! 668.2414607646324   49658.666124994445
ERR! 668.2414607646324   49658.666124994445
ERR! 668.2414607646324   49658.6

ERR! 668.2414607646324   49658.666124994445
ERR! 668.2414607646324   49658.666124994445
ERR! 668.2414607646324   49658.666124994445
ERR! 668.2414607646324   49658.666124994445
ERR! 668.2414607646324   49658.666124994445
ERR! 668.2414607646324   49658.666124994445
ERR! 668.2414607646324   49658.666124994445
ERR! 668.2414607646324   49658.666124994445
ERR! 668.2414607646324   49658.666124994445
ERR! 668.2414607646324   49658.666124994445
ERR! 668.2414607646324   49658.666124994445
ERR! 668.2414607646324   49658.666124994445
ERR! 668.2414607646324   49658.666124994445
ERR! 668.2414607646324   49658.666124994445
ERR! 668.2414607646324   49658.666124994445
ERR! 668.2414607646324   49658.666124994445
ERR! 668.2414607646324   49658.666124994445
ERR! 668.2414607646324   49658.666124994445
ERR! 668.2414607646324   49658.666124994445
ERR! 668.2414607646324   49658.666124994445
ERR! 668.2414607646324   49658.666124994445
ERR! 668.2414607646324   49658.666124994445
ERR! 668.2414607646324   49658.6

ERR! 668.2414607646324   49658.666124994445
ERR! 668.2414607646324   49658.666124994445
ERR! 668.2414607646324   49658.666124994445
ERR! 668.2414607646324   49658.666124994445
ERR! 668.2414607646324   49658.666124994445
ERR! 668.2414607646324   49658.666124994445
ERR! 668.2414607646324   49658.666124994445
ERR! 668.2414607646324   49658.666124994445
ERR! 668.2414607646324   49658.666124994445
ERR! 668.2414607646324   49658.666124994445
ERR! 668.2414607646324   49658.666124994445
ERR! 668.2414607646324   49658.666124994445
ERR! 668.2414607646324   49658.666124994445
ERR! 668.2414607646324   49658.666124994445
ERR! 668.2414607646324   49658.666124994445
ERR! 668.2414607646324   49658.666124994445
ERR! 668.2414607646324   49658.666124994445
ERR! 668.2414607646324   49658.666124994445
ERR! 668.2414607646324   49658.666124994445
ERR! 668.2414607646324   49658.666124994445
ERR! 668.2414607646324   49658.666124994445
ERR! 668.2414607646324   49658.666124994445
ERR! 668.2414607646324   49658.6

ERR! 668.2414607646324   49658.666124994445
ERR! 668.2414607646324   49658.666124994445
ERR! 668.2414607646324   49658.666124994445
ERR! 668.2414607646324   49658.666124994445
ERR! 668.2414607646324   49658.666124994445
ERR! 668.2414607646324   49658.666124994445
ERR! 668.2414607646324   49658.666124994445
ERR! 668.2414607646324   49658.666124994445
ERR! 668.2414607646324   49658.666124994445
ERR! 668.2414607646324   49658.666124994445
ERR! 668.2414607646324   49658.666124994445
ERR! 668.2414607646324   49658.666124994445
ERR! 668.2414607646324   49658.666124994445
ERR! 668.2414607646324   49658.666124994445
ERR! 668.2414607646324   49658.666124994445
ERR! 668.2414607646324   49658.666124994445
ERR! 668.2414607646324   49658.666124994445
ERR! 668.2414607646324   49658.666124994445
ERR! 668.2414607646324   49658.666124994445
ERR! 668.2414607646324   49658.666124994445
ERR! 668.2414607646324   49658.666124994445
ERR! 668.2414607646324   49658.666124994445
ERR! 668.2414607646324   49658.6

KeyboardInterrupt: 